In [1]:
import os
import sys
import json
import glob

In [ ]:
def get_results(result_dir, test_file):

    result_files = glob.glob(f"{result_dir}/*.jsonl") # path to gem generated jsonl files

    result_dict = {}
    for result_file in result_files:
        chunk_id = result_file.split("/")[-1].split(".")[0].split("-")[0]
        result_dict[chunk_id] = result_file

    chunks = [f'chunk_{i}' for i in range(8)]
    all_results = []
    for chunk_id in chunks:
        with open(result_dict[chunk_id], "r", encoding="utf-8") as f:
            for line in f:
                data = json.loads(line)
                all_results.append(data)

    all_results_dict = {}
    for result in all_results:
        all_results_dict[result["question_id"]] = result

    with open(test_file, "r", encoding="utf-8") as f:
        test_data = json.load(f)

    final_results = []
    for data in test_data:
        new_data = {}   
        idx = data["id"]
        try:
            new_data["id"] = idx
            new_data["ecg"] = data["ecg"]
            new_data["image"] = data["image"]
            new_data["GEM_generated"] = all_results_dict[idx]["text"]
            new_data["GPT4o_generated"] = data["conversations"][1]["value"]
            new_data["machine_measurements"] = data["machine_measurements"]
            new_data["report"] = data["report"]
            new_data["model_id"] = all_results_dict[idx]["model_id"]

            final_results.append(new_data)

        except:
            raise Exception(f"idx {idx} not found")

    return final_results

In [ ]:
grounding_test_file = "...path_to/ecg-grounding-test.json" # path to test files

result_dir = "../eval_outputs/xxx/xxx" # path to gem generated jsonl files

In [5]:
final_results = get_results(result_dir, grounding_test_file)

In [ ]:
save_name = "gem7b_results.json"

save_dir = os.path.join("grounding_model_outputs/raw_results", save_name) # save the combined generated files

with open(save_dir, "w") as f:
    json.dump(final_results, f, indent=4)